In [0]:
from pyspark.sql.functions import col

dim_airports = (
    spark.read.table("flights_project.silver.airports_silver")
    .where(col("__END_AT").isNull())
    .select(
        "airport_sk",
        "airport_id",
        "airport_name",
        "city",
        "country"
    )
)

dim_flights = (
    spark.read.table("flights_project.silver.flights_silver")
    .where(col("__END_AT").isNull())
    .select(
        "flight_sk",
        "flight_id",
        "flight_date",
        "airline",
        "origin",
        "destination"
    )
)

dim_passengers = (
    spark.read.table("flights_project.silver.passengers_silver")
    .where(col("__END_AT").isNull())
    .select(
        "passenger_sk",
        "passenger_id",
        "name",
        "gender",
        "nationality"
    )
)

fact_bookings = (
    spark.read.table("flights_project.silver.bookings_silver")
    .select(
        "booking_id",
        "passenger_id",
        "flight_id",
        "airport_id",
        "amount",
        "booking_date"
    )
)

In [0]:
flights_business_join = (
    fact_bookings
    .join(dim_passengers, "passenger_id", "left")
    .join(dim_flights, "flight_id", "left")
    .join(dim_airports, "airport_id", "left")
)

In [0]:
dim_airports.write.mode("overwrite").saveAsTable("flights_project.gold.dim_airports")
dim_flights.write.mode("overwrite").saveAsTable("flights_project.gold.dim_flights")
dim_passengers.write.mode("overwrite").saveAsTable("flights_project.gold.dim_passengers")
fact_bookings.write.mode("overwrite").saveAsTable("flights_project.gold.fact_bookings")
flights_business_join.write.mode("overwrite").saveAsTable("flights_project.gold.flights_business_join")